# This jupyter notebook loads de csv prices data and estimate the total price of the project 

In [1]:
%run load_csv.py

### Load the data and set the sema format

In [74]:
import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd

base_path = "/Users/Curi/DATAKNOWN/Prueba Técnica Cientifico de Datos/Prueba tecnica 1/Datos/"

X = load_csv_data(path = base_path + "X.csv", separation = ",", decimal = ".", reversed = True)
Y = load_csv_data(path = base_path + "Y.csv", separation = ";", decimal = ",", reversed = True)
Y["Date"] = pd.to_datetime(Y["Date"], format="%d/%m/%Y")
Y["Date"] = pd.to_datetime(Y["Date"], format="%Y-%m-%d")
Z = load_csv_data(path = base_path + "Z.csv", separation = ",", decimal = ".", reversed = False)

### Transform the Date into numerical values to perform the mathematical analysis easier

In [75]:
total_data = X["Date"].count()
max_date = X["Date"].iloc[-1]
min_date = X["Date"].iloc[0]
total_days_sampled = (max_date-min_date).days

X["Date_Num"] = total_days_sampled - (max_date-X["Date"]).dt.days
Y["Date_Num"] = total_days_sampled - (max_date-Y["Date"]).dt.days
Z["Date_Num"] = total_days_sampled - (max_date-Z["Date"]).dt.days

### Plot the raw data

In [76]:
fig, ax = plt.subplots(1,3,figsize=(30, 7))
ax[0].plot(X["Date"],X["Price"])
ax[1].plot(Y["Date"],Y["Price"])
ax[2].plot(Z["Date"],Z["Price"])

for a in ax:
    a.tick_params(axis='both', which='major', labelsize=15)

for k in range(3):
    ax[k].set_xlabel("Date", fontsize=25)
ax[0].set_ylabel("Price", fontsize=25)

ax[0].set_title("X", fontsize=25)
ax[1].set_title("Y", fontsize=25)
ax[2].set_title("Z", fontsize=25)

filename = 'raw_data.png'
plt.savefig(filename, dpi=300, bbox_inches='tight')
plt.close(fig)